In [1]:
import os
import json
from datetime import date
from statistics import median
from file import load_all

In [2]:
dir = 'option_leverage'
os.makedirs(dir, exist_ok=True)

for symbol, data in load_all().items():
  details = data['ticker_details']
  leverage_history = {}
  for w in data['options_weekly']:
    today = w['day'].date
    for p in w['options']:
      c = p['contract']
      expiration_date = date.fromisoformat(c.expiration_date)
      if expiration_date.isoweekday() == 5:
        expiration_weeks = round((expiration_date - today).days / 7)
        if expiration_weeks > 0 and p['history'] and p['history'][0].date == today:
          call_price = p['history'][0].close
          lvr = c.strike_price / call_price
          if expiration_weeks not in leverage_history:
            leverage_history[expiration_weeks] = []
          leverage_history[expiration_weeks].append(lvr)

  leverage = {w: median(v) for w, v in leverage_history.items()}

  # normalize values so longer expirations have lower leverage
  leverage = {k: v for k, v in zip(sorted(leverage.keys()), sorted(leverage.values(), reverse=True))}

  print(f'{symbol}: {len(leverage)} expirations')
  with open(f'{dir}/{symbol}.json', 'w') as json_file:
    json.dump(leverage, json_file, indent=4, sort_keys=True)

print(f'Call leverages saved under {dir}')

USD: 32 expirations
SMH: 101 expirations
TSLA: 143 expirations
QQQ: 145 expirations
AMZN: 151 expirations
NFLX: 138 expirations
MSFT: 153 expirations
SNOW: 119 expirations
SHOP: 122 expirations
IHI: 46 expirations
NET: 132 expirations
GOOGL: 153 expirations
SPY: 155 expirations
MAGS: 16 expirations
SOXL: 122 expirations
AAPL: 151 expirations
TQQQ: 122 expirations
SQ: 124 expirations
CRM: 124 expirations
WDAY: 79 expirations
NOW: 95 expirations
FNGS: 25 expirations
NVDA: 154 expirations
Call leverages saved under option_leverage
